In [1]:
import sys
# !{sys.executable} -m spacy download en
import re, numpy as np, pandas as pd
from pprint import pprint

# Gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora       
from nltk.stem import WordNetLemmatizer as lemmatize

from gensim.utils import  simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt

In [2]:
# NLTK Stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])
stop_words.extend(['app', 'barclays','need','will','pay'])
%matplotlib inline
warnings.filterwarnings("ignore",category=DeprecationWarning)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [34]:
import pandas as pd
import numpy as np
import json, os, uuid
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv("/content/drive/MyDrive/BarclaysReviews/final_merged_reviews.csv")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
ratingColName = 'customer_sentiment'
df[ratingColName] = 'POSITIVE'
for i in df.index:
    if df['rating'][i] > 3:
      df[ratingColName][i] = 'POSITIVE'
    else:
      df[ratingColName][i] = 'NEGATIVE'
df = df[df['review_description'].notnull() & df[ratingColName].str.contains('NEGATIVE')]

<ipython-input-35-a85d3a9c182c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[ratingColName][i] = 'NEGATIVE'
<ipython-input-35-a85d3a9c182c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[ratingColName][i] = 'POSITIVE'


In [33]:

replace_dict = {"google pay":"googlepay","contactless card":"contactlesscrd","credit card":"credicard","debit card":"debitcard"}
df['review_description'] = df['review_description'].str.lower()
df['review_description'].replace(replace_dict,inplace=True,regex=True)
df['review_description'] = df['review_description'].str.lower()
df[df["review_description"].str.contains("google")][["review_description", "rating",ratingColName]] 

,review_description,rating,customer_sentiment
159,used to work very well but is now useless. i c...,1,NEGATIVE
200,just spoke to a barclays customer service rep....,1,NEGATIVE
243,still unable to use contactless. i want to use...,1,NEGATIVE
295,now the contactless mobile doesn't work and it...,1,NEGATIVE
334,app is keep on crashing in galaxy s22 and goog...,1,NEGATIVE
...,...,...,...
29707,will barclays end up supporting googlepay?,3,NEGATIVE
29866,looks like an app from 2007.. even worse. it c...,1,NEGATIVE
29945,"upd 2022.02.10 can't tell, they don't do anyth...",3,NEGATIVE
30293,won't allow me to use googlepay. it seems that...,3,NEGATIVE


In [36]:
def sent_to_words(sentences):
    for sent in sentences:
        sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
        sent = re.sub('\s+', ' ', sent)  # remove newline chars
        sent = re.sub("\'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent)  

# Convert to list
data = df.review_description.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1])

[['now', 'this', 'is', 'awful', 'because', 'cant', 'make', 'payments', 'with', 'my', 'phone', 'but', 'then', 'every', 'time', 'open', 'the', 'app', 'get', 'prompted', 'to', 'change', 'settings', 'so', 'that', 'can', 'make', 'payments', 'with', 'my', 'phone', 'and', 'after', 'ive', 'done', 'that', 'it', 'tells', 'me', 'that', 'cant', 'make', 'payments', 'with', 'my', 'phone', 'every', 'time', 'after', 'that', 'when', 'just', 'want', 'to', 'use', 'the', 'app', 'it', 'holds', 'everything', 'up', 'with', 'prompt', 'that', 'cant', 'make', 'payments', 'with', 'my', 'phone']]


In [37]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import spacy 
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
nlp.Defaults.stop_words.update(['app', 'barclays','need','will'])
nlp.Defaults.stop_words.update(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])
len(nlp.Defaults.stop_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


348

In [41]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# !python3 -m spacy download en  # run in terminal once
def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    #nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    i =0
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        #print(doc)
        #texts_out.append([token.lemma_ for token in doc ])
         #[wnl.lemmatize(token.lemma_.lower()) for token in nlp(text) if not token.is_stop and not token.is_punct]
        texts_out.append([token.lemma_ for token in doc if  not token.is_punct ])
        print(texts_out[i])
        i = i+1
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
    return texts_out

data_ready = process_words(data_words)  # processed Text Data!

Streaming output truncated to the last 5000 lines.
['let', 'straight', 'allow', 'user', 'set', 'contactless', 'mobile', 'payment', 'use', 'google', 'samsung', 'pay', 'someone', 'think', 'idea', 'restrict', 'contactless', 'payment', 'unnecessarily', 'restrictive', 'hassle', 'do', 'not', 'work', 'despite', 'work', 'past', 'totally', 'software', 'design']
['since', 'late', 'update', 'contactless', 'payment', 'android', 'phone', 'long', 'work', 'speak', 'say', 'yes']
['earth', 'long', 'mobile', 'payment', 'device']
['notoriously', 'unreliable', 'contactless', 'mobile', 'feature', 'regularly', 'turn', 'require', 'set', 'time', 'chat', 'feature', 'crash', 'constantly', 'version', 'smart', 'device', 'positive', 'side', 'intuitive', 'allow', 'access', 'information', 'need']
['disappoint', 'bad', 'experience', 'ever', 'bank']
['update', 'today', 'say', 'can', 'not', 'contactless', 'payment', 'device', 'use', 'along', 'abandon', 'shopping', 'half', 'way', 'home', 'completely', 'useless', 'let', 

In [14]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [11]:
!pip install lda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.9/320.9 kB 24.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 16.0 MB/s eta 0:00:00
  Created wheel for lda: filename=lda-2.0.0-cp310-cp310-linux_x86_64.whl size=628204 sha256=0369636bc8e4fd363e21257d8ac02f08731698f72d4329ac755c3f228a0cf94a
  Stored in directory: /root/.cache/pip/wheels/9d/32/fa/07fdf278a0b24269ce4d0be3d642c28dd0e888a4c5f2031279
Successfully built lda


In [42]:
# Create Dictionary
id2word = corpora.Dictionary(data_ready)

# Create Corpus: Term Document Frequency
corpus = [id2word.doc2bow(text) for text in data_ready]

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=10,
                                           alpha='symmetric',
                                           iterations=1000,
                                           per_word_topics=True)

pprint(lda_model.print_topics())

[(0,
  '0.051*"close" + 0.044*"ask" + 0.039*"day" + 0.037*"allow" + 0.034*"issue" + '
  '0.033*"transfer" + 0.033*"way" + 0.033*"frustrating" + 0.026*"nothing" + '
  '0.024*"however"'),
 (1,
  '0.111*"account" + 0.061*"chat" + 0.038*"option" + 0.036*"log" + '
  '0.028*"message" + 0.023*"much" + 0.022*"able" + 0.020*"business" + '
  '0.019*"register" + 0.017*"error"'),
 (2,
  '0.083*"transaction" + 0.044*"service" + 0.042*"show" + 0.038*"credit" + '
  '0.034*"download" + 0.034*"statement" + 0.026*"balance" + 0.026*"simple" + '
  '0.023*"date" + 0.023*"debit"'),
 (3,
  '0.000*"complicate" + 0.000*"awfully" + 0.000*"intutitive" + '
  '0.000*"authenticate" + 0.000*"responsive" + 0.000*"itsnot" + '
  '0.000*"complecated" + 0.000*"registered" + 0.000*"beneficial" + '
  '0.000*"beach"'),
 (4,
  '0.000*"sd" + 0.000*"yye" + 0.000*"ug" + 0.000*"xzt" + 0.000*"te" + '
  '0.000*"yeuyam" + 0.000*"dxxt" + 0.000*"fight" + 0.000*"cyty" + 0.000*"gr"'),
 (5,
  '0.058*"one" + 0.050*"new" + 0.039*"detail" 

In [18]:
warnings.warn('Error: A warning just appeared')
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_ready)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(10)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
pd.set_option('display.max_colwidth', -1)
df_dominant_topic.head(10)


In [ ]:
len(df_dominant_topic)

In [ ]:
# Display setting to show more characters in column
pd.options.display.max_colwidth = 100

sent_topics_sorteddf_mallet = pd.DataFrame()
sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=False).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Representative Text"]

# Show
sent_topics_sorteddf_mallet.head(10)

In [ ]:
from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import matplotlib.colors as mcolors

cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]  # more colors: 'mcolors.XKCD_COLORS'

cloud = WordCloud(stopwords=stop_words,
                  background_color='white',
                  width=2500,
                  height=1800,
                  max_words=10,
                  colormap='tab10',
                  color_func=lambda *args, **kwargs: cols[i],
                  prefer_horizontal=1.0)

topics = lda_model.show_topics(formatted=False)

fig, axes = plt.subplots(3, 2, figsize=(10,10), sharex=True, sharey=True)

for i, ax in enumerate(axes.flatten()):
    fig.add_subplot(ax)
    topic_words = dict(topics[i][1])
    cloud.generate_from_frequencies(topic_words, max_font_size=300)
    plt.gca().imshow(cloud)
    plt.gca().set_title('Topic ' + str(i), fontdict=dict(size=16))
    plt.gca().axis('off')


plt.subplots_adjust(wspace=0, hspace=0)
plt.axis('off')
plt.margins(x=0, y=0)
plt.tight_layout()
plt.show()

In [ ]:
from collections import Counter
topics = lda_model.show_topics(formatted=False)
data_flat = [w for w_list in data_ready for w in w_list]
counter = Counter(data_flat)

out = []
for i, topic in topics:
    for word, weight in topic:
        out.append([word, i , weight, counter[word]])

df = pd.DataFrame(out, columns=['word', 'topic_id', 'importance', 'word_count'])        

# Plot Word Count and Weights of Topic Keywords
fig, axes = plt.subplots(3, 2, figsize=(16,10), sharey=True, dpi=160)
cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]
for i, ax in enumerate(axes.flatten()):
    ax.bar(x='word', height="word_count", data=df.loc[df.topic_id==i, :], color=cols[i], width=0.5, alpha=0.3, label='Word Count')
    ax_twin = ax.twinx()
    ax_twin.bar(x='word', height="importance", data=df.loc[df.topic_id==i, :], color=cols[i], width=0.2, label='Weights')
    ax.set_ylabel('Word Count', color=cols[i])
    ax_twin.set_ylim(0, 0.030); ax.set_ylim(0, 3500)
    ax.set_title('Topic: ' + str(i), color=cols[i], fontsize=16)
    ax.tick_params(axis='y', left=False)
    ax.set_xticklabels(df.loc[df.topic_id==i, 'word'], rotation=30, horizontalalignment= 'right')
    ax.legend(loc='upper left'); ax_twin.legend(loc='upper right')

fig.tight_layout(w_pad=2)    
fig.suptitle('Word Count and Importance of Topic Keywords', fontsize=22, y=1.05)    
plt.show()


In [ ]:
pip install "pandas<2.0.0"

In [ ]:
pip install pyLDAvis

In [ ]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

In [ ]:
corpus